#### Name：Experimental_Data_export
#### People：Ke Liu

In [1]:
import subprocess as sp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import seaborn as sns
from statannot import add_stat_annotation
from statannotations.Annotator import Annotator

In [2]:
def intersect(a,b,command,result):
    _shell2call = '/home/qukun/liuke/miniconda3/envs/eccDNA/bin/bedtools intersect -nonamecheck -a {0} -b {1} {2} > {3}'.format(a, b, command, result)
    sp.check_call(_shell2call, shell=True)
    return result

In [3]:
all_methods = ['ATAC','DNAT' ,'DSAT', 'DNARCAT', 'DNARCAU', 'DSARCAT', 'DSARCAU', 'DNRT7', 'DSRT7', 'Nanopore_WGS', 'NGS_WGS' ]
pub_methods = ['DNRT7', 'DNARCAU', 'DSRT7', 'DSAT', 'ATAC', 'Nanopore_WGS', 'NGS_WGS' ]
NGS_methods = ['DNARCAU', 'DSAT', 'NGS_WGS']
Nanopore_methods = ['DNRT7', 'DSRT7', 'Nanopore_WGS']

In [4]:
NGS_path = '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/filtered/'
ATAC_path = '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/filtered/'
Nanopore_path = '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/filtered/'

In [100]:
expansion_bed1 = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion1.bed'
expansion_bed2 = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion2.bed'
expansion_bed3 = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion3.bed'
expansion_bed12 = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion12.bed'
expansion_bed123 = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion123.bed'

In [101]:
Nanopore_cnv1 = pd.read_csv('/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/Nanopore_WGS1.bam_CNVs',sep = '\t',names=['chr','start','end','num','CNV'])
Nanopore_cnv1.chr = Nanopore_cnv1.apply(lambda x: 'chr'+str(x.chr),axis=1)
Nanopore_cnv1 = Nanopore_cnv1[Nanopore_cnv1.CNV=='gain']
Nanopore_cnv1.to_csv(expansion_bed1,sep='\t',index=None,header=None)

In [102]:
Nanopore_cnv2 = pd.read_csv('/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/Nanopore_WGS2.bam_CNVs',sep = '\t',names=['chr','start','end','num','CNV'])
Nanopore_cnv2.chr = Nanopore_cnv2.apply(lambda x: 'chr'+str(x.chr),axis=1)
Nanopore_cnv2 = Nanopore_cnv2[Nanopore_cnv2.CNV=='gain']
Nanopore_cnv2.to_csv(expansion_bed2,sep='\t',index=None,header=None)

In [103]:
Nanopore_cnv3 = pd.read_csv('/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/Nanopore_WGS3.bam_CNVs',sep = '\t',names=['chr','start','end','num','CNV'])
Nanopore_cnv3.chr = Nanopore_cnv3.apply(lambda x: 'chr'+str(x.chr),axis=1)
Nanopore_cnv3 = Nanopore_cnv3[Nanopore_cnv3.CNV=='gain']
Nanopore_cnv3.to_csv(expansion_bed3,sep='\t',index=None,header=None)

In [104]:
intersect(expansion_bed1, expansion_bed2, '', expansion_bed12)
intersect(expansion_bed12, expansion_bed3, '', expansion_bed123)

'/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion123.bed'

In [ ]:
total_table = pd.DataFrame(index=['{0}{1}'.format(a, b) for a in pub_methods for b in [1,2,3]],columns=['simple eccDNA', 'simple ecDNA', '<2kb', '2kb-10kb', '>10kb', 'ec<2kb', 'ec2kb-10kb', 'ec>10kb', 'complex', 'complex_diff', 'complex_ecDNA', 'chim<2kb', 'chim2kb-10kb', 'chim>10kb',  'chimec<2kb', 'chimec2kb-10kb', 'chimec>10kb', 'cpg','partialgene','fullgene','exon','intron'])

In [125]:
total_table = pd.DataFrame(index=['{0}{1}'.format(a, b) for a in pub_methods for b in [1,2,3]],columns=['eccDNA', 'ecDNA', '<2kb', '2kb-10kb', '>10kb', 'ec<2kb', 'ec2kb-10kb', 'ec>10kb', 'complex', 'complex_diff', 'complex_ecDNA', 'chim<2kb', 'chim2kb-10kb', 'chim>10kb',  'chimec<2kb', 'chimec2kb-10kb', 'chimec>10kb', 'cpg','partialgene','fullgene','exon','intron'])

## 1 Identify Copy Number Amplication Region using Nanopore_WGS data

In [169]:
ratio_table = pd.DataFrame(columns=pub_methods,index=['1','2','3'])
ecDNA_count_table = pd.DataFrame(columns=pub_methods,index=['1','2','3'])
namp_count_table = pd.DataFrame(columns=pub_methods,index=['1','2','3'])
for sample in ['{0}{1}'.format(a, b) for a in Nanopore_methods for b in [1,2,3]]:
        result_bed = Nanopore_path+sample+'.bed'
        ecDNA_bed = '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/ecDNA/'+sample+'.bed'
        intersect(result_bed, expansion_bed123, '-wa -wb', ecDNA_bed)
        locals()[sample+'_ecDNA'] = pd.read_csv(ecDNA_bed,sep='\t',header=None)
        locals()[sample] = pd.read_csv(result_bed,sep='\t',header=None)
        ratio_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample+'_ecDNA'][3].unique())/len(locals()[sample][3].unique())
        ecDNA_count_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample+'_ecDNA'][3].unique())
        namp_count_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample][3].unique()) - len(locals()[sample+'_ecDNA'][3].unique())
        total_table.loc[sample,'eccDNA'] = len(locals()[sample][3].unique())
        total_table.loc[sample,'ecDNA'] = len(locals()[sample+'_ecDNA'][3].unique())

In [170]:
for sample in ['{0}{1}'.format(a, b) for a in NGS_methods for b in [1,2,3]]:
        expansion_bed = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion.bed'
        result_bed = NGS_path+sample+'.bed'
        ecDNA_bed = '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/ecDNA/'+sample+'.bed'
        intersect(result_bed, expansion_bed123, '-wa -wb', ecDNA_bed)
        locals()[sample+'_ecDNA'] = pd.read_csv(ecDNA_bed,sep='\t',header=None)
        locals()[sample] = pd.read_csv(result_bed,sep='\t',header=None)
        ratio_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample+'_ecDNA'][3].unique())/len(locals()[sample][3].unique())
        ecDNA_count_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample+'_ecDNA'][3].unique())
        namp_count_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample][3].unique()) - len(locals()[sample+'_ecDNA'][3].unique())
        total_table.loc[sample,'eccDNA'] = len(locals()[sample][3].unique())
        total_table.loc[sample,'ecDNA'] = len(locals()[sample+'_ecDNA'][3].unique())

In [171]:
for sample in ['ATAC1','ATAC2','ATAC3']:
        expansion_bed = '/home/qukun/liuke/workspace/benchmark/result/cons/cnv/cnv/expansion.bed'
        result_bed = ATAC_path+sample+'.bed'
        ecDNA_bed = '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/ecDNA/'+sample+'.bed'
        intersect(result_bed, expansion_bed123, '-wa -wb', ecDNA_bed)
        locals()[sample+'_ecDNA'] = pd.read_csv(ecDNA_bed,sep='\t',header=None)
        locals()[sample] = pd.read_csv(result_bed,sep='\t',header=None)
        ratio_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample+'_ecDNA'][3].unique())/len(locals()[sample][3].unique())
        ecDNA_count_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample+'_ecDNA'][3].unique())
        namp_count_table.loc[sample[-1:],sample[:-1]]=len(locals()[sample][3].unique()) - len(locals()[sample+'_ecDNA'][3].unique())
        total_table.loc[sample,'eccDNA'] = len(locals()[sample][3].unique())
        total_table.loc[sample,'ecDNA'] = len(locals()[sample+'_ecDNA'][3].unique())

## 2 Length Bin

In [172]:
bins = [0, 2*1e3, 1e4, 1e7]
labels = ['<2kb', '2kb-10kb', '>10kb'] 

In [173]:
for sample in ['{0}{1}'.format(a, b) for a in Nanopore_methods for b in [1,2,3]]:
    result_txt = '/home/qukun/liuke/workspace/benchmark/result/3rd/raw/' + sample + '.txt' 
    locals()[sample+'_merge'] = pd.read_csv(result_txt,sep='\t')
    locals()[sample+'_merge'] = locals()[sample+'_merge'][locals()[sample+'_merge']['eccdna_status']=='cyclic']
    locals()[sample+'_merge'].insert(7,'len_region',pd.cut(locals()[sample+'_merge'].merge_len, bins, labels = labels))
    j = 0
    for i in locals()[sample+'_merge'].len_region.value_counts().index:
        total_table.loc[sample,i] = locals()[sample+'_merge'].len_region.value_counts().loc[i]
        total_table.loc[sample,'ec'+i] = locals()[sample+'_merge'][locals()[sample+'_merge'].id.isin(locals()[sample+'_ecDNA'][3].unique())].len_region.value_counts().loc[i]
        total_table.loc[sample,'chim'+i] = locals()[sample+'_merge'][locals()[sample+'_merge'].num_region>1].len_region.value_counts().loc[i]
        total_table.loc[sample,'chimec'+i] = locals()[sample+'_merge'][(locals()[sample+'_merge'].id.isin(locals()[sample+'_ecDNA'][3].unique()))&(locals()[sample+'_merge'].num_region>1)].len_region.value_counts().loc[i]
        j=j+1

In [174]:
for sample in ['{0}{1}'.format(a, b) for a in NGS_methods for b in [1,2,3]]:
    locals()[sample].insert(7,'len_region',pd.cut(locals()[sample][6], bins, labels = labels))
    j = 0
    for i in locals()[sample].len_region.value_counts().index:
        length_count.loc[j] = [i, sample, sample[:-1], locals()[sample].len_region.value_counts()[i]]
        total_table.loc[sample,i] = locals()[sample].len_region.value_counts().loc[i]
        total_table.loc[sample,'ec'+i] = locals()[sample][locals()[sample][3].isin(locals()[sample+'_ecDNA'][3].unique())].len_region.value_counts().loc[i]
        total_table.loc[sample,'chim'+i] = 0
        total_table.loc[sample,'chimec'+i] = 0
        j=j+1

In [175]:
for sample in ['ATAC1','ATAC2','ATAC3']:
    locals()[sample].insert(7,'len_region',pd.cut(locals()[sample][6], bins, labels = labels))
    for i in locals()[sample].len_region.value_counts().index:
        length_count.loc[j] = [i, sample, sample[:-1], locals()[sample].len_region.value_counts()[i]]
        total_table.loc[sample,i] = locals()[sample].len_region.value_counts().loc[i]
        total_table.loc[sample,'ec'+i] = locals()[sample][locals()[sample][3].isin(locals()[sample+'_ecDNA'][3].unique())].len_region.value_counts().loc[i]
        total_table.loc[sample,'chim'+i] = 0
        total_table.loc[sample,'chimec'+i] = 0
        j=j+1

## 3 Chimeric Proportion

In [176]:
sum_table = pd.DataFrame(columns=['total','same','different'])

In [177]:
for sample in ['{0}{1}'.format(a, b) for a in Nanopore_methods for b in [1,2,3]]:
    locals()[sample+'_complex_list'] = list(locals()[sample][3].value_counts()[locals()[sample][3].value_counts()>1].index)
    locals()[sample+'_complex_bed'] = locals()[sample][locals()[sample][3].isin(locals()[sample+'_complex_list'])]
    i=0
    for ecc in locals()[sample+'_complex_list']:
        if (len(locals()[sample+'_complex_bed'][locals()[sample+'_complex_bed'][3]==ecc][0].unique())==1):
            i+=1
    total_table.loc[sample,'complex'] = len(locals()[sample+'_complex_list'])
    total_table.loc[sample,'complex_diff'] = len(locals()[sample+'_complex_list'])-i
    total_table.loc[sample,'complex_ecDNA'] = len(set(locals()[sample+'_ecDNA'][3]).intersection(set(locals()[sample+'_complex_list'])))
    sum_table.loc[sample] = [len(locals()[sample+'_complex_list']),i,len(locals()[sample+'_complex_list'])-i]

In [178]:
for sample in ['{0}{1}'.format(a, b) for a in NGS_methods for b in [1,2,3]] + ['ATAC1','ATAC2','ATAC3']:
    total_table.loc[sample,'complex'] = 0
    total_table.loc[sample,'complex_diff'] = 0
    total_table.loc[sample,'complex_ecDNA'] = 0

## 4 Gene & Gene region

In [179]:
def intersect(a,b,rate1,rate2,command,result):
    _shell2call = '/home/qukun/liuke/miniconda3/envs/eccDNA/bin/bedtools intersect -nonamecheck -a {0} -b {1} -f {2} -F {3} {4} > {5}'.format(a, b,rate1,rate2, command, result                                                                                                                                                  )
    sp.check_call(_shell2call, shell=True)
    return result

In [180]:
cpg_bed = '/home/qukun/liuke/reference/UCSC/hg38/annotation/cpg_ucsc.bed'
exon_bed = '/home/qukun/liuke/reference/UCSC/hg38/annotation/exon.bed'
intron_bed = '/home/qukun/liuke/reference/UCSC/hg38/annotation/intron.bed'
gene_bed = '/home/qukun/liuke/reference/UCSC/hg38/annotation/gene.bed'
num=[1,2,3]
NGS_list = ['{0}{1}'.format(a,b) for a in NGS_methods for b in num]
Nanopore_list = ['{0}{1}'.format(a,b) for a in Nanopore_methods for b in num]
for sample in NGS_list:
    ecc_bed = '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/filtered/'+sample+'.bed'
    intersect(exon_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.exon.bed')
    intersect(intron_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.intron.bed')
    intersect(cpg_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.cpg.bed')
    intersect(gene_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.fullgene.bed')
    intersect(gene_bed, ecc_bed, 1e-9, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.partialgene.bed')
    for element in ['cpg','partialgene','fullgene','exon','intron']:
        if (open('/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.'+element+'.bed').readlines()==[]):
            total_table.loc[sample,element]=0
        else:
            locals()[sample+'_'+element] = pd.read_csv('/home/qukun/liuke/workspace/benchmark/result/2nd/circle-map/processed/annotation/'+sample+'.'+element+'.bed',sep='\t',header=None)
            total_table.loc[sample,element]=len(locals()[sample+'_'+element][10].unique())

In [181]:
for sample in Nanopore_list:    
    ecc_bed = '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/filtered/'+sample+'.bed'
    intersect(exon_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.exon.bed')
    intersect(intron_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.intron.bed')
    intersect(cpg_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.cpg.bed')
    intersect(gene_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.fullgene.bed')
    intersect(gene_bed, ecc_bed, 1e-9, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.partialgene.bed')
    for element in ['cpg','partialgene','fullgene','exon','intron']:
        if (open('/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.'+element+'.bed').readlines()==[]):
            total_table.loc[sample,element]=0
        else:
            locals()[sample+'_'+element] = pd.read_csv('/home/qukun/liuke/workspace/benchmark/result/3rd/processed/annotation/'+sample+'.'+element+'.bed',sep='\t',header=None)
            total_table.loc[sample,element]=len(locals()[sample+'_'+element][10].unique())

In [182]:
for sample in ['ATAC1','ATAC2','ATAC3']:    
    ecc_bed = '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/filtered/'+sample+'.bed'
    intersect(exon_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.exon.bed')
    intersect(intron_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.intron.bed')
    intersect(cpg_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.cpg.bed')
    intersect(gene_bed, ecc_bed, 1, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.fullgene.bed')
    intersect(gene_bed, ecc_bed, 1e-9, 1e-9, '-wb', '/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.partialgene.bed')
    for element in ['cpg','partialgene','fullgene','exon','intron']:
        if (open('/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.'+element+'.bed').readlines()==[]):
            total_table.loc[sample,element]=0
        else:
            locals()[sample+'_'+element] = pd.read_csv('/home/qukun/liuke/workspace/benchmark/result/2nd/circle_finder/processed/annotation/'+sample+'.'+element+'.bed',sep='\t',header=None)
            total_table.loc[sample,element]=len(locals()[sample+'_'+element][10].unique())

In [183]:
total_table.to_csv('~/summary_20230725.csv')